In [1]:
import csv
from pathlib import Path
from fast_forward.ranking import Ranking
from fast_forward.index import InMemoryIndex
from fast_forward.encoder import TCTColBERTQueryEncoder as TCTColBERTQueryEncoderFF
from ir_measures import read_trec_qrels, calc_aggregate, nDCG, RR, R, AP, P
from numpy import arange
import csv

c:\Users\Revi\.conda\envs\custom_fast_forward\lib\site-packages\scipy\__init__.py:173: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.19.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
c:\Users\Revi\.conda\envs\custom_fast_forward\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
sparse_ranking_2019 = Ranking.from_file(Path("../index/dev/testing/msmarco-passage-test2019-sparse10000.txt"))
sparse_ranking_2019.cut(5000)
sparse_ranking_2020 = Ranking.from_file(Path("../index/dev/testing/msmarco-passage-test2020-sparse10000.txt"))
sparse_ranking_2020.cut(5000)

index = InMemoryIndex.from_disk("../index/dev/testing/ffindex_passage_2019_2020.pkl",encoder=TCTColBERTQueryEncoderFF("castorini/tct_colbert-msmarco"))

Some weights of the model checkpoint at castorini/tct_colbert-msmarco were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
with open(
    "../index/dev/testing/msmarco-test2019-queries.tsv",
    encoding="utf-8",
    newline=""
) as fp:
    queries = {q_id: q for q_id, q in csv.reader(fp, delimiter="\t")}
print(f"loaded {len(queries)} queries")

alpha = arange(0,1.01,0.05)
eta = range(5,106,5)
result_cc = index.get_scores(
    sparse_ranking_2019,
    queries,
    method = "CC",
    alpha = alpha,
    cutoff=10,
    early_stopping=False
)
result_rrf = index.get_scores(
    sparse_ranking_2019,
    queries,
    method = "RRF",
    eta = eta,
    cutoff=10,
    early_stopping=False
)

loaded 200 queries
custom FFI version


100%|██████████| 43/43 [00:00<00:00, 56.82it/s]


w:  0.0
w:  0.05
w:  0.1
w:  0.15000000000000002
w:  0.2
w:  0.25
w:  0.30000000000000004
w:  0.35000000000000003
w:  0.4
w:  0.45
w:  0.5
w:  0.55
w:  0.6000000000000001
w:  0.65
w:  0.7000000000000001
w:  0.75
w:  0.8
w:  0.8500000000000001
w:  0.9
w:  0.9500000000000001
w:  1.0
custom FFI version


100%|██████████| 43/43 [00:00<00:00, 60.50it/s]


w:  5
w:  10
w:  15
w:  20
w:  25
w:  30
w:  35
w:  40
w:  45
w:  50
w:  55
w:  60
w:  65
w:  70
w:  75
w:  80
w:  85
w:  90
w:  95
w:  100
w:  105


In [48]:
qrels = list(read_trec_qrels("../index/dev/testing/2019qrels-pass.txt"))
with open(r'2019_test_results.csv','a') as f1: # need "a" and not w to append to a file, if not will overwrite
    writer=csv.writer(f1, delimiter=',',lineterminator='\n',)
    row = ["fusion_function","dataset","alpha","R@10","AP@10","P@10","nDCG@10","RR(rel=2)@10"]
    writer.writerow(row)
    for a in alpha:
        f = calc_aggregate([R@10,AP@10,P@10,nDCG@10, RR(rel=2)@10], qrels, result_cc[a].run)
        row = ["CC","test2019",a,f[R@10],f[AP@10],f[P@10],f[nDCG@10],f[RR(rel=2)@10]]
        writer.writerow(row)

with open(r'2019_test_results.csv','a') as f1: # need "a" and not w to append to a file, if not will overwrite
    writer=csv.writer(f1, delimiter=',',lineterminator='\n',)
    for n in eta:
        f = calc_aggregate([R@10,AP@10,P@10,nDCG@10, RR(rel=2)@10], qrels, result_rrf[n].run)
        row = ["RRF","test2019",n,f[R@10],f[AP@10],f[P@10],f[nDCG@10],f[RR(rel=2)@10]]
        writer.writerow(row)